# Generación y exportación de primitivas 3D
## Herramientas: vedo, trimesh, open3d, numpy

In [ ]:
# Instalación (si es necesario)
!pip install vedo trimesh open3d numpy

In [ ]:
import numpy as np
import vedo
import trimesh
import open3d as o3d
import json, csv, os

## Crear puntos en el espacio

In [ ]:
# Lista de coordenadas
coords = [
    [0, 0, 0],
    [2, 0, 0],
    [0, 2, 0],
    [0, 0, 2],
    [2, 2, 2]
]

## Crear primitivas 3D con variación de color y forma

In [ ]:
# Lista para almacenar los objetos
vedo_objs = []

for i, coord in enumerate(coords):
    if i % 3 == 0:
        obj = vedo.Cube(pos=coord, side=1).color('red')  # corregido aquí
    elif i % 3 == 1:
        obj = vedo.Sphere(pos=coord, r=0.5).color('green')
    else:
        obj = vedo.Cylinder(pos=coord, r=0.3, height=1).color('blue')
    vedo_objs.append(obj)

## Exportar con vedo (.obj)

In [ ]:
# Escena combinada y exportación .obj
scene = vedo.Assembly(vedo_objs)
vedo.write(scene, "scene_vedo.obj")

## Exportar con trimesh (.stl)

In [ ]:
trimesh_objs = []

for vobj in vedo_objs:
    mesh_tri = vedo.Mesh(vobj).triangulate()  # Garantiza que las caras sean triángulos
    vertices = mesh_tri.points
    faces = mesh_tri.faces()
    mesh = trimesh.Trimesh(vertices=vertices, faces=faces, process=False)
    trimesh_objs.append(mesh)

# Escena y exportación
scene = trimesh.Scene(trimesh_objs)
scene.export("scene_trimesh.stl")



## Exportar con Open3D (.ply)

In [ ]:
# Tomamos el objeto vedo y lo triangulamos para asegurar que todas las caras tengan 3 vértices
mesh_tri = vedo.Mesh(vobj).triangulate()

# Convertimos a formato NumPy adecuado
vertices = np.asarray(mesh_tri.points, dtype=np.float64)
faces = np.asarray(mesh_tri.faces(), dtype=np.int32)

# Creamos malla Open3D
mesh_o3d = o3d.geometry.TriangleMesh()
mesh_o3d.vertices = o3d.utility.Vector3dVector(vertices)
mesh_o3d.triangles = o3d.utility.Vector3iVector(faces)
mesh_o3d.compute_vertex_normals()

# Exportamos
o3d.io.write_triangle_mesh("mesh_open3d.ply", mesh_o3d)


## Bonus: Leer coordenadas desde archivo .csv

In [ ]:
# Guardar ejemplo .csv
with open("coords.csv", "w", newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["x", "y", "z"])
    writer.writerows(coords)

# Leer coordenadas desde .csv
coords_csv = []
with open("coords.csv", "r") as f:
    reader = csv.DictReader(f)
    for row in reader:
        coords_csv.append([float(row['x']), float(row['y']), float(row['z'])])

print("Coordenadas leídas:", coords_csv)